# <font color='darkblue'>Jane Street Market Prediction</font>
### This is an extensive data analysis for the jane street market dataset, this notebook will go through the train and features csv's for an extensive exploratory data analysis, Also some data cleaning and preprocessing will be done along the way.
#### Please note that this is a relatively large notebook with a lot of exhaustive analytics as it meant to be that way to give a comprehensive understanding of the dataset so it might take a considerable amount of time to run ( around 30 minutes) so be patient.
## [The EDA](#eda) will be devided to two main parts:
      
## 1- [General EDA](#general)
>#### **Which will include the following:**
  >- [Resp Data Analysis](#resp)
  >- [Date](#date)
  >- [Weight](#weight)

## 2 - [Features Data Analysis](#features)
>#### **Which will include the following:**
  >- [Null Values](#nulls)
  >- [Cumulative growth](#growth)
  >- [Multicollinearity](#multicollinearity)
  >- [Outliers](#outlier)
  >- [Feature 0](#f0)

<h1><center>Let's dive right in!</center></h1>

![](https://media1.tenor.com/images/ed3ccde29b0efef4a88e13353f6923ba/tenor.gif)



### Importing Libraries 

In [ ]:
import os
import gc
import sys

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline
from IPython.core.display import display, HTML
sns.set_style('darkgrid')

# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
# from sklearn.linear_model import LogisticRegression as log
# from sklearn.model_selection import train_test_split as split 
# from sklearn.model_selection import GridSearchCV as Grid
# from sklearn.naive_bayes import GaussianNB as GNB
# from sklearn.svm import SVC
# from sklearn.linear_model import LinearRegression as linear
# from sklearn.linear_model import SGDClassifier as SGD
# import xgboost as xgb
# from sklearn.decomposition import IncrementalPCA as ipca
# from sklearn.metrics import (roc_auc_score, precision_score, recall_score, f1_score,
#                              confusion_matrix, accuracy_score, roc_curve, auc)

In [ ]:
#!pip install seaborn --upgrade

In [ ]:
#!pip install plotly --upgrade

## **Loading Data**

In [ ]:
df = pd.read_csv('../input/jane-street-market-prediction/train.csv')

In [ ]:
pd.set_option('display.max_columns', 140)

In [ ]:
#saving the original dataset length
org_len = len(df)
df.head(2)

In [ ]:
df.info()

#### Trying to cut corners to save some memory

In [ ]:
df.feature_0 = df.feature_0.astype(np.int8)
df.date= df.date.astype(np.int16)
df.ts_id = df.ts_id.astype(np.int32)

#### The following code loop through the dataframe to change float64 columns to float32 only if there is a really few amount of data (a very conservative threshold of 0.1%) between -.0001:.0001 to avoid hurting accuracy of small values columns

In [ ]:
for i in df:
    if df[i].dtype == np.float64:
        if (((df[i] < .0001) & (df[i] > -.0001)).mean()) > .001:
            print(i)

In [ ]:
for i in df:
    if df[i].dtype == np.float64:
        if (((df[i] < .0001) & (df[i] > -.0001)).mean()) < .001:
            df[i] = df[i].astype(np.float32)
            gc.collect();

In [ ]:
df.info()

> ### we can see a noticeable difference in memory usage

### Assuring that Data is stored by date

In [ ]:
df.sort_values(by= ['date','ts_id'],inplace=True)

### Adding target

In [ ]:
df['action'] = np.where(df['resp'] > 0,1,0)
df.action = df.action.astype(np.int8)

<a id="eda"></a>
# **EDA**

<a id='general'></a>
# **General Data Analysis**

<a id='resp'></a>
## **Resp Data Analysis** 

In [ ]:
fig = plt.figure(figsize=(16,6))
ax = plt.subplot(1,1,1)
df.groupby('date')[['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']].sum().cumsum().plot(ax=ax)
plt.title('Cumulative Sum of Different RESP\'s',fontsize=18)
plt.xlabel('Date',fontsize=14)
plt.legend(fontsize=12);

> ### It can be noticed the there were more gain in the first 100 days, We can also notice that resp_4 has the highest cumulative sum on the other hand resp_1 has the smallest cumulative sum.

### Now we plot the average of each Resp

In [ ]:
fig = px.line(df.groupby('date')[['resp_1', 'resp_2', 'resp_3', 'resp_4','resp']].mean(),
              x= df.groupby('date')[['resp_1', 'resp_2', 'resp_3', 'resp_4','resp']].mean().index,
              y= ['resp_1', 'resp_2', 'resp_3', 'resp_4','resp'],
              title= '\naverage Resp per day')
fig.layout.xaxis.title = 'Day' 
fig.layout.yaxis.title = 'Avg Resp'
fig.show()

In [ ]:
fig,((ax11,ax12,ax13),(ax21,ax22,ax23),(ax31,ax32,ax33),(ax41,ax42,ax43),(ax51,ax52,ax53)) = plt.subplots(5,3,figsize=(20,24))
plt.subplots_adjust(hspace=0.35)
ax11.hist(df.resp,bins=150)
ax11.axvline(df.resp.mean()+df.resp.std(),color='darkorange',alpha=.8)
ax11.axvline(df.resp.mean()-df.resp.std(),color='darkorange',alpha=.8)
df.resp.plot.hist(bins= 150,ax=ax12)
ax12.axvline(df.resp.mean()+df.resp.std(),color='darkorange',alpha=.8)
ax12.axvline(df.resp.mean()-df.resp.std(),color='darkorange',alpha=.8)
ax12.set_xlim(-.08,.08)
ax13.hist(df.resp,bins=150)
ax13.set_yscale('log')
skew= round(df.resp.skew(),4)
kurt= round(df.resp.kurtosis())
std1= round((((df.resp.mean()-df.resp.std()) < df.resp ) & (df.resp < (df.resp.mean()+df.resp.std()))).mean()*100,2)
props = dict(boxstyle='round', facecolor='white', alpha=0.5)
ax11.text(.02,.96,'μ = {}\nstd = {}\nskewness = {}\nkurtosis = {}\n% values in 1 std = {}%'.format(round(df.resp.mean(),4),round(df.resp.std(),4),skew,kurt,std1),
         transform=ax11.transAxes, verticalalignment='top',bbox=props,fontsize=10)
ax11.set_title('Resp Hist Normal scale',fontsize=14)
ax12.set_title('Resp Hist normal scale zoomed',fontsize=14)
ax13.set_title('Resp Hist with freq on a log scale',fontsize=14);
ax11.set_xlabel('')
ax11.set_ylabel('')
ax12.set_xlabel('')
ax12.set_ylabel('')
ax13.set_xlabel('')
ax13.set_ylabel('')
ax21.hist(df.resp_1,bins=150)
ax21.axvline(df.resp_1.mean()+df.resp_1.std(),color='darkorange',alpha=.8)
ax21.axvline(df.resp_1.mean()-df.resp_1.std(),color='darkorange',alpha=.8)
df.resp_1.plot.hist(bins= 150,ax=ax22)
ax22.axvline(df.resp_1.mean()+df.resp_1.std(),color='darkorange',alpha=.8)
ax22.axvline(df.resp_1.mean()-df.resp_1.std(),color='darkorange',alpha=.8)
ax22.set_xlim(-.08,.08)
ax23.hist(df.resp_1,bins=150)
ax23.set_yscale('log')
skew= round(df.resp_1.skew(),4)
kurt= round(df.resp_1.kurtosis())
std1= round((((df.resp_1.mean()-df.resp_1.std()) < df.resp_1 ) & (df.resp_1 < (df.resp_1.mean()+df.resp_1.std()))).mean()*100,2)
ax21.text(.02,.96,'μ = {}\nstd = {}\nskewness = {}\nkurtosis = {}\n% values in 1 std = {}%'.format(round(df.resp_1.mean(),4),round(df.resp_1.std(),4),skew,kurt,std1),
         transform=ax21.transAxes, verticalalignment='top',bbox=props,fontsize=10)
ax21.set_title('Resp_1 Hist Normal scale',fontsize=14)
ax22.set_title('Resp_1 Hist normal scale zoomed',fontsize=14)
ax23.set_title('Resp_1 Hist with freq on a log scale',fontsize=14);
ax21.set_xlabel('')
ax21.set_ylabel('')
ax22.set_xlabel('')
ax22.set_ylabel('')
ax23.set_xlabel('')
ax23.set_ylabel('')
ax31.hist(df.resp_2,bins=150)
ax31.axvline(df.resp_2.mean()+df.resp_2.std(),color='darkorange',alpha=.8)
ax31.axvline(df.resp_2.mean()-df.resp_2.std(),color='darkorange',alpha=.8)
df.resp_2.plot.hist(bins= 150,ax=ax32)
ax32.axvline(df.resp_2.mean()+df.resp_2.std(),color='darkorange',alpha=.8)
ax32.axvline(df.resp_2.mean()-df.resp_2.std(),color='darkorange',alpha=.8)
ax32.set_xlim(-.08,.08)
ax33.hist(df.resp_2,bins=150)
ax33.set_yscale('log')
skew= round(df.resp_2.skew(),4)
kurt= round(df.resp_2.kurtosis())
std1= round((((df.resp_2.mean()-df.resp_2.std()) < df.resp_2 ) & (df.resp_2 < (df.resp_2.mean()+df.resp_2.std()))).mean()*100,2)
ax31.text(.02,.96,'μ = {}\nstd = {}\nskewness = {}\nkurtosis = {}\n% values in 1 std = {}%'.format(round(df.resp_2.mean(),4),round(df.resp_2.std(),4),skew,kurt,std1),
         transform=ax31.transAxes, verticalalignment='top',bbox=props,fontsize=10)
ax31.set_title('Resp_2 Hist Normal scale',fontsize=14)
ax32.set_title('Resp_2 Hist normal scale zoomed',fontsize=14)
ax33.set_title('Resp_2 Hist with freq on a log scale',fontsize=14);
ax31.set_xlabel('')
ax31.set_ylabel('')
ax32.set_xlabel('')
ax32.set_ylabel('')
ax33.set_xlabel('')
ax33.set_ylabel('')
ax41.hist(df.resp_3,bins=150)
ax41.axvline(df.resp_3.mean()+df.resp_3.std(),color='darkorange',alpha=.8)
ax41.axvline(df.resp_3.mean()-df.resp_3.std(),color='darkorange',alpha=.8)
df.resp_3.plot.hist(bins= 150,ax=ax42)
ax42.axvline(df.resp_3.mean()+df.resp_3.std(),color='darkorange',alpha=.8)
ax42.axvline(df.resp_3.mean()-df.resp_3.std(),color='darkorange',alpha=.8)
ax42.set_xlim(-.08,.08)
ax43.hist(df.resp_3,bins=150)
ax43.set_yscale('log')
skew= round(df.resp_3.skew(),4)
kurt= round(df.resp_3.kurtosis())
std1= round((((df.resp_3.mean()-df.resp_3.std()) < df.resp_3 ) & (df.resp_3 < (df.resp_3.mean()+df.resp_3.std()))).mean()*100,2)
ax41.text(.02,.96,'μ = {}\nstd = {}\nskewness = {}\nkurtosis = {}\n% values in 1 std = {}%'.format(round(df.resp_3.mean(),4),round(df.resp_3.std(),4),skew,kurt,std1),
         transform=ax41.transAxes, verticalalignment='top',bbox=props,fontsize=10)
ax41.set_title('Resp_3 Hist Normal scale',fontsize=14)
ax42.set_title('Resp_3 Hist normal scale zoomed',fontsize=14)
ax43.set_title('Resp_3 Hist with freq on a log scale',fontsize=14);
ax41.set_xlabel('')
ax41.set_ylabel('')
ax42.set_xlabel('')
ax42.set_ylabel('')
ax43.set_xlabel('')
ax43.set_ylabel('')
ax51.hist(df.resp_4,bins=150)
ax51.axvline(df.resp_4.mean()+df.resp_4.std(),color='darkorange',alpha=.8)
ax51.axvline(df.resp_4.mean()-df.resp_4.std(),color='darkorange',alpha=.8)
df.resp_4.plot.hist(bins= 150,ax=ax52)
ax52.axvline(df.resp_4.mean()+df.resp_4.std(),color='darkorange',alpha=.8)
ax52.axvline(df.resp_4.mean()-df.resp_4.std(),color='darkorange',alpha=.8)
ax52.set_xlim(-.08,.08)
ax53.hist(df.resp_4,bins=150)
ax53.set_yscale('log')
skew= round(df.resp_4.skew(),4)
kurt= round(df.resp_4.kurtosis())
std1= round((((df.resp_4.mean()-df.resp_4.std()) < df.resp_4 ) & (df.resp_4 < (df.resp_4.mean()+df.resp_4.std()))).mean()*100,2)
ax51.text(.02,.96,'μ = {}\nstd = {}\nskewness = {}\nkurtosis = {}\n% values in 1 std = {}%'.format(round(df.resp_4.mean(),4),round(df.resp_4.std(),4),skew,kurt,std1),
         transform=ax51.transAxes, verticalalignment='top',bbox=props,fontsize=10)
ax51.set_title('Resp_4 Hist Normal scale',fontsize=14)
ax52.set_title('Resp_4 Hist normal scale zoomed',fontsize=14)
ax53.set_title('Resp_4 Hist with freq on a log scale',fontsize=14)
ax51.set_xlabel('')
ax51.set_ylabel('')
ax52.set_xlabel('')
ax52.set_ylabel('')
ax53.set_xlabel('')
ax53.set_ylabel('')
fig.suptitle('RESPs Historgrams on Different Scales',fontsize=18,y=.92);

In [ ]:
sns.pairplot(df[['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']],corner=True);

we can see that Resp is highly related to Resp_4
also Resp_1 and Resp_2 are highly relted to each other 
from the relation shown in this figure and the standard deviation and distribution shown in the figure before this one we can assume that Resp is more related to longer time horizon invest as longer time horizon are associated with more return and higher risk

### [The Basics of Investment Time Horizons](https://www.investopedia.com/terms/t/timehorizon.asp)
> ####  An Investment Time Horizon is the period where one expects to hold an investment for a specific goal. Investments are generally broken down into two main categories: stocks (riskier) and bonds (less risky). The longer the Time Horizon, the more aggressive, or riskier portfolio, an investor can build. The shorter the Time Horizon, the more conservative, or less risky, the investor may want to adopt. 

[source: investopedia.com](https://www.investopedia.com/terms/t/timehorizon.asp)

<a id='date'></a>
## **Date**

In [ ]:
df.date.unique()

> ### The Date seem to contain 2 years of trading data since the trading days of the year are approximately 252 : 253 days
https://en.wikipedia.org/wiki/Trading_day

In [ ]:
fig = px.area(data_frame= df.groupby('date')[['resp']].count(),title='Number of operation per day')
fig.update_traces( showlegend = False)
fig.layout.xaxis.title = 'Day' 
fig.layout.yaxis.title = 'Number of operations'
fig.show()

### We will create a moving standard deviation of 20 days (which is a month of trading) for the average of resp

In [ ]:
date_df = df.groupby('date')[['resp']].mean()
std20 = []
for i in range(len(date_df)):
    if i <20:
        std20.append(np.nan)
    else:
        moving_std = date_df['resp'][i-20:i].std()
        std20.append(moving_std)
date_df['moving_std'] = std20
date_df.tail(2)

In [ ]:
fig = px.line(data_frame=date_df,y=['resp','moving_std'],title='Average Resp & 20 day moving standard deviation')
fig.layout.xaxis.title = 'Day' 
fig.layout.yaxis.title = 'Avg Resp'
fig.show()

### Now we check the standard deviation of each resp for each day

In [ ]:
fig, (ax1,ax2) = plt.subplots(2,1,figsize=(14,12))
df.groupby('date')[['resp_1', 'resp_2', 'resp_3', 'resp_4']].std().plot(ax=ax1,color=['steelblue','darkorange','red','green'],alpha=.8)
df.groupby('date')[['resp_1', 'resp_2', 'resp_3', 'resp_4']].std().plot.kde(ax=ax2)
fig.suptitle('Resp\'s Std',fontsize=18,y=.96)
ax2.set_xlabel('')
ax1.set_xlabel('')
ax2.set_title('kde of each resp std', fontsize=14)
ax1.set_title('std of Resp\'s for each trading day',fontsize=14);

> ### As it was mentioned before the standard deviation seems to increase with resp mostly related to longer time horizon investments
> ### It can also be noticed that the deviation was kinda higher in the first 100 days as it was mentioned by many kagglers that there may was some kind of trading model adjustment done after the 80th day.

<a id='weight'></a>
## **Weight**

In [ ]:
df.weight[df.weight !=0].describe()

In [ ]:
fig = plt.figure(figsize=(18,7))
grid =  gridspec.GridSpec(2,3,figure=fig,hspace=.3,wspace=.2)
ax1 = fig.add_subplot(grid[0, 0])
ax2 = fig.add_subplot(grid[0, 1])
ax3 = fig.add_subplot(grid[1, 0])
ax4 = fig.add_subplot(grid[1, 1])
ax5 = fig.add_subplot(grid[:, 2])
sns.boxplot(x = df.weight,width=.5,ax=ax1)
ax2.hist(df.weight, color='#404788ff',alpha=.6, bins= list([-.05] + list(10**np.arange(-2,2.24,.05))))
ax2.set_xscale('symlog')
ax2.set_xlim(-.05,227)
sns.boxplot(x = df.weight[df.weight != 0],width=.5,ax=ax3)
ax1.set_title('Weights including zero weights',fontsize=14)
ax3.set_title('Weights not including zero weights',fontsize=14)
ax2.set_title('Weights including zero weights (log)',fontsize=14)
ax4.set_title('Weights not including zero weights (log)',fontsize=14)
props = dict(boxstyle='round', facecolor='white', alpha=0.4)
ax1.text(.2,.9,'μ = {}    std = {}\nmin = {}    max = {}'.format(round(df.weight.mean(),3),round(df.weight.std(),3),round(df.weight.min(),3),round(df.weight.max(),3)),
         transform=ax1.transAxes, verticalalignment='top',bbox=props,fontsize=12)
ax3.text(.2,.9,'μ = {}        std = {}\nmin = {}    max = {}'.format(round(df.weight[df.weight != 0].mean(),3),round(df.weight[df.weight != 0].std(),3),
                                                              round(df.weight[df.weight != 0].min(),3),round(df.weight[df.weight != 0].max(),3)),
         transform=ax3.transAxes, verticalalignment='top',bbox=props,fontsize=12)
ax4.hist(df.weight[df.weight !=0],color='#404788ff',alpha=.6,bins=10**np.arange(-2.16,2.24,.05))
ax4.set_xscale('log')
ax4.set_xticks((.01,.03,.1,.3,1,3,10,30,100))
ax4.set_xticklabels((.01,.03,.1,.3,1,3,10,30,100))
ax5.pie(((df.weight==0).mean(),(1-(df.weight==0).mean())),startangle=300,wedgeprops=dict(width=0.5),
        labels=('Zeros\n{}%'.format(round((df.weight==0).mean()*100,2)),'Nonzeros\n{}%'.format(round((1-(df.weight==0).mean())*100,2))),
        textprops={'fontsize': 12},colors=['#404788ff','#55c667ff'])
ax5.set_title('Zeros vs non-zero weights',fontsize=14)
ax1.set_xlabel('')
ax2.set_xlabel('')
ax3.set_xlabel('')
ax2.set_ylabel('')
ax5.set_ylabel('')
ax4.set_xlabel('');

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle('Nonzero weights histogram in different scales',fontsize=18)
ax1 = plt.subplot(3,1,1)
ax1.hist(df.weight[df.weight !=0],color='darkblue',alpha=.7, bins=10**np.arange(-2.16,2.23,.05))
plt.xscale('log')
plt.xticks((.01,.03,.1,.3,1,3,10,30,100),(.01,.03,.1,.3,1,3,10,30,100))
ax2 = plt.subplot(3,1,2)
sns.distplot(df.weight[df.weight != 0], color='darkblue', bins=400, ax=ax2) 
ax3 = plt.subplot(3,1,3)
ax3.hist(df.weight[(df.weight !=0) & (df.weight < 3.197 )],color='darkblue',alpha=.7, bins=200)
ax3.set_xlim(0,3.3)
ax2.set_xlabel('') 
ax1.set_title('All values (log-scale)',fontsize=14)
ax2.set_title('kde of the distribution',fontsize=14)
ax3.set_title('75% of the Values',fontsize=14)
plt.subplots_adjust(hspace=.4);

### Now we take a deeper look at outliers

In [ ]:
fig, (ax1,ax2) = plt.subplots(2,1,figsize=(16,8))
fig.suptitle('Weight outliers',fontsize=18)
sns.boxplot(df.weight,width=.5, ax=ax1)
ax1.axvline(np.percentile(df.weight,95), color= 'green',label='95.0%',linestyle=':',linewidth=3)
ax1.axvline(np.percentile(df.weight,99), color= 'darkblue',label='99.0%',linestyle=':',linewidth=3)
ax1.axvline(np.percentile(df.weight,99.9), color= 'darkorange',label='99.9%',linestyle=':',linewidth=3)
ax1.axvline(np.percentile(df.weight,99.99), color= 'magenta',label='99.99%',linestyle=':',linewidth=3)
ax1.legend(fontsize=13)
sns.boxplot(df.weight[df.weight !=0],width=.5, ax=ax2)
ax2.axvline(np.percentile(df.weight[df.weight !=0],95), color= 'green',label='95.0%',linestyle=':',linewidth=3)
ax2.axvline(np.percentile(df.weight[df.weight !=0],99), color= 'darkblue',label='99.0%',linestyle=':',linewidth=3)
ax2.axvline(np.percentile(df.weight[df.weight !=0],99.9), color= 'darkorange',label='99.9%',linestyle=':',linewidth=3)
ax2.axvline(np.percentile(df.weight[df.weight !=0],99.99), color= 'magenta',label='99.99%',linestyle=':',linewidth=3)
ax2.legend(fontsize=13)
ax1.set_title('All weights', fontsize= 14)
ax2.set_title('Non-zero weights', fontsize= 14)
ax1.set_xlabel('')
ax2.set_xlabel('');

In [ ]:
sns.scatterplot(data=df, x='resp',y='weight', color= 'blue', alpha=.3)
plt.title('Resp vs Weight\ncorrelation={}'.format(round(df.weight.corr(df.resp),4)));

> ### We can see that weight is not linearly correlated with Resp but it's clear that higher weight are only associated with low Resp values

<a id='features'></a>
# Features data analysis

### Loading the features csv

In [ ]:
df_f =  pd.read_csv('../input/jane-street-market-prediction/features.csv')
df_f.head(5)

In [ ]:
fig = px.bar(df_f.set_index('feature').T.sum(), title='Number of tags for each feature')
fig.layout.xaxis.tickangle = 300
fig.update_traces( showlegend = False)
fig.layout.xaxis. dtick = 5
fig.layout.xaxis.title = ''
fig.layout.yaxis.title = ''
fig.show()

<a id='nulls'></a>
## **Exploring the Null values**

In [ ]:
fig = px.bar(x = df.isnull().sum().index,y= df.isnull().sum().values,title= 'Number of Null values')
fig.layout.xaxis.tickangle = 300
fig.layout.xaxis. dtick = 5
fig.layout.yaxis. dtick = 100000
fig.layout.xaxis.title = ''
fig.layout.yaxis.title = ''
fig.layout.xaxis.showgrid = True
fig.show()

#### Finding out the features with missing values more than 10 %

In [ ]:
0.1 * len(df)

In [ ]:
nulls = df.isnull().sum()
nulls_list = list(nulls[(nulls >239049)].index)
nulls_list

#### **Looking into the relationship between these features since there is kind of pattern in the number of null values**

In [ ]:
df[nulls_list].corr().style.background_gradient(cmap='viridis')

### Since the number of null values in these columns are huge (larger than one quarter of a million!) we will be dropping nulls with more than 10% null values

In [ ]:
df.drop(columns=nulls_list,inplace=True)

### now concerning the remaining nulls we will look firstly to the coefficient of variation

In [ ]:
(df.iloc[:,7:-2].std() / df.iloc[:,7:-2].mean()).head(5)

#### the coefficient of variation seems unreliable due to the value of the mean being near to zero

### Now we can take a bird's-eye view  of features distributions

In [ ]:
df.iloc[:,7:-2].hist(bins=100,figsize=(20,74),layout=(29,4));

### **The following code will make a grid of horizontal box plot with the mean ploted too to get a comprehensive solid understanding of the features distributions**
> #### please note that we used customized 0.1%:99.9% whisker to show extreme outliers since the data is strongly centered.

In [ ]:
fig = plt.figure(figsize=(20,80))
grid =  gridspec.GridSpec(29,4,figure=fig,hspace=.5,wspace=.05)
featstr = [i for i in df.columns[7:-2]]
counter = 0
for i in range(29):
    for j in range(4):
        subf = fig.add_subplot(grid[i, j]);
        sns.boxplot(x= df[featstr[counter]],saturation=.5,color= 'blue', ax= subf,width=.5,whis=(.1,99.9));
        subf.axvline(df[featstr[counter]].mean(),color= 'darkorange')
        subf.set_xlabel('')
        subf.set_title('{}'.format(featstr[counter]),fontsize=14)
        counter += 1
        gc.collect();

### We can see that there is a lot of outliers affecting the distribution of each feature.
### Also since the majority of values are heavily centerd around the mean, we will fill null values using the mean.

In [ ]:
df.mean().reset_index().T

In [ ]:
df.fillna(df.mean(axis=0),inplace=True)

<a id='growth'></a>
## Features growth 

In [ ]:
fig = plt.figure(figsize=(18,24))
grid =  gridspec.GridSpec(7,5,figure=fig,hspace=.3,wspace=.3)
counter = 0
f = featstr[1:36]
for i in range(7):
    for j in range(5):
        subf = fig.add_subplot(grid[i, j]);
        cumsum = df[f[counter]].cumsum()
        subf.plot(cumsum,color= 'darkblue', alpha=.7);
        subf.set_xlabel('')
        subf.set_ylabel('')
        subf.set_title('{}'.format(f[counter]),fontsize=14)
        counter += 1
        del cumsum
        gc.collect();

In [ ]:
gc.collect()

In [ ]:
fig = plt.figure(figsize=(18,24))
grid =  gridspec.GridSpec(7,5,figure=fig,hspace=.3,wspace=.3)
counter = 0
f = featstr[36:71]
for i in range(7):
    for j in range(5):
        subf = fig.add_subplot(grid[i, j]);
        cumsum = df[f[counter]].cumsum()
        subf.plot(cumsum,color= 'darkblue', alpha=.7);
        subf.set_xlabel('')
        subf.set_ylabel('')
        subf.set_title('{}'.format(f[counter]),fontsize=14)
        counter += 1
        del cumsum
        gc.collect();

In [ ]:
fig = plt.figure(figsize=(18,24))
grid =  gridspec.GridSpec(7,5,figure=fig,hspace=.3,wspace=.3)
counter = 0
f = featstr[71:106]
for i in range(7):
    for j in range(5):
        subf = fig.add_subplot(grid[i, j]);
        cumsum = df[f[counter]].cumsum()
        subf.plot(cumsum,color= 'darkblue', alpha=.7);
        subf.set_xlabel('')
        subf.set_ylabel('')
        subf.set_title('{}'.format(f[counter]),fontsize=14)
        counter += 1
        del cumsum
        gc.collect();

In [ ]:
fig = plt.figure(figsize=(18,6))
grid =  gridspec.GridSpec(2,5,figure=fig,hspace=.3,wspace=.3)
counter = 0
f = featstr[106:]
for i in range(2):
    for j in range(5):
        subf = fig.add_subplot(grid[i, j]);
        cumsum = df[f[counter]].cumsum()
        subf.plot(cumsum,color= 'darkblue', alpha=.7);
        subf.set_xlabel('')
        subf.set_ylabel('')
        subf.set_title('{}'.format(f[counter]),fontsize=14)
        counter += 1
        del cumsum
        gc.collect();

> ### Many features seem to be linearly growing but some features like 81, 82, and 83 are actually decreasing, and there are some features that fluctuate 

<a id='multicollinearity'></a>
## Correlation between features

#### First we make a correlation dataframe

In [ ]:
corr = df.iloc[:,7:-2].corr()

#### Plotting a heatmap for features correlation

In [ ]:
fig = plt.figure(figsize=(18,12))
ax = plt.subplot(1,1,1)
sns.heatmap(corr,ax= ax, cmap='coolwarm');

#### Now we check pearson correlation coefficient values for all features

In [ ]:
corr.style.background_gradient(cmap='coolwarm')

>### **It looks like there is a lot of multicollinearity between features and also it looks like there is a pattern of couples in the features space and this pattern is kinda broke at some features  like feature_41**

In [ ]:
featstr2 = [ i for i in featstr if i not in ['feature_41','feature_64']]
len(featstr)

In [ ]:
fig = plt.figure(figsize=(22,44))
grid =  gridspec.GridSpec(12,5,figure=fig,hspace=.5,wspace=.2)
counter = 1
for i in range(12):
    for j in range(5):
        if counter == 113:
            break
        subf = fig.add_subplot(grid[i, j]);
        sns.scatterplot(x= df[featstr2[counter]], y = df[featstr2[counter+1]], ax= subf);
        cor = round(df[featstr2[counter]].corr(df[featstr2[counter+1]]) * 100,2)
        subf.set_xlabel('')
        subf.set_ylabel('')
        subf.set_title('{} & {}\nCorrelation = {}%'.format(featstr2[counter],featstr2[counter+1],cor),fontsize=14)
        counter += 2
        gc.collect();  

> ### As expected since these are finance related features a lot of features are highly correlate with each other

### Now we explore highly correlated groups of features

#### We start off with featrues: **[feature_19, feature_20, feature_21, feature_22, feature_23, feature_24, feature_25, feature_26, feature_29]** since there is kind of multicollinearity cluster

In [ ]:
plt.figure(figsize=(12,6)) 
sns.heatmap(df[featstr2[15:23]].corr(),center=0,cmap='coolwarm',annot=True,cbar=False);

In [ ]:
sns.pairplot(df[featstr2[15:23]],corner=True);

> ### Despite the fact that Pearson coefficients of correlation are really high between these features the relationships are not completely linear, also it can be noticed that the outliers affect the shape of scatter plots.

### Now we check the correlation again but between other group 

In [ ]:
plt.figure(figsize=(12,6)) 
sns.heatmap(df[featstr2[23:31]].corr(),center=0,cmap='coolwarm',annot=True,cbar=False);

In [ ]:
sns.pairplot(df[featstr2[23:31]],corner=True);

> ### things looks like kinda the same with the other cluster, it  also worth mentioning that both of these features clusters are negatively correlated with each other.

In [ ]:
plt.figure(figsize=(18,6)) 
sns.heatmap(df[featstr2[15:31]].corr(),center=0,cmap='coolwarm',annot=True,cbar=False);

> <a id='outlier'></a>
## Outliers

#### First we take a look at the mean of features.

In [ ]:
fig = px.bar(df[featstr].mean(), title='Features mean values')
fig.layout.xaxis.tickangle = 300
fig.update_traces(showlegend = False)
fig.layout.xaxis. dtick = 5
fig.layout.xaxis.title = ''
fig.layout.yaxis.title = ''
fig.show()

In [ ]:
fig = px.bar(df[featstr].max(), title='Features Max Values')
fig.layout.xaxis.tickangle = 300
fig.update_traces(showlegend = False)
fig.layout.xaxis. dtick = 5
fig.layout.xaxis.title = ''
fig.layout.yaxis.title = ''
fig.show()

In [ ]:
fig = px.bar(df[featstr].min(), title='Features Min Values')
fig.layout.xaxis.tickangle = 300
fig.update_traces(showlegend = False)
fig.layout.xaxis. dtick = 5
fig.layout.xaxis.title = ''
fig.layout.yaxis.title = ''
fig.show()

In [ ]:
fig, (ax1,ax2,ax3)= plt.subplots(3,1,figsize=(10,12))
plt.subplots_adjust(hspace=.3)
sns.distplot(df[featstr].max(),ax= ax1 )
sns.distplot(df[featstr].min(),ax= ax2)
sns.distplot(df[featstr].mean(),ax= ax3)
fig.suptitle('distribution of mean max and min for features',fontsize=16)
ax1.set_title('distribution  of features max values',fontsize=14)
ax1.text(.82,.56,'std = {}'.format(round(df[featstr].max().std(),2)),transform=ax1.transAxes, verticalalignment='top',bbox=props,fontsize=12)
ax2.set_title('distribution  of features min values',fontsize=14)
ax2.text(.82,.56,'std = {}'.format(round(df[featstr].min().std(),2)),transform=ax2.transAxes, verticalalignment='top',bbox=props,fontsize=12)
ax3.set_title('distribution  of features mean values',fontsize=14)
ax3.text(.82,.56,'std = {}'.format(round(df[featstr].mean().std(),2)),transform=ax3.transAxes, verticalalignment='top',bbox=props,fontsize=12);

> ### We can see that despite the fact the mean values are not that different from each other the min and max values are very deviated with highly skewed distribution

In [ ]:
gc.collect()

### Now we do more statistically oriented exploring to outliers

In [ ]:
for i in featstr[1:]:
    print('{}\n0.1%:99.9% are between: {}\nmax: {}\nmin: {}\n75% are under: {}'.format(i,np.percentile(df[i],(.1,99.9)), df[i].max(),df[i].min(),np.percentile(df[i],75)),
         '\n===============================')

In [ ]:
df[(df.feature_56== df.feature_56.max())|(df.feature_57== df.feature_57.max())|(df.feature_58== df.feature_58.max()) | (df.feature_59== df.feature_59.max())]

> ### It can be inferred that the dataset has extreme outliers it also worth mentioning that some of outliers are accompanied with large values from neighbor columns which is a result of high multicollinearity between features

### Now we will make a bold move by removing these extreme outliers that are above 99.9% of feature data

#### To avoid removing more data while looping through the data set we will make a list of 99.9% mark for each and every single feature
#### We will also create a list for negative outliers values "using .1 % mark" to be explored later 

In [ ]:
n999 = [ np.percentile(df[i],99.9) for i in featstr[1:]]
n001 = [ np.percentile(df[i],.1) for i in featstr[1:]]

#### Now we iterate over the dataset to remove outliers

In [ ]:
for i, j in enumerate(featstr[1:]):
    df = df[df[j] < n999[i]]
    gc.collect()

### Finding the ratio of the data lost in removing the outliers

In [ ]:
str(round(((org_len - len(df))/org_len)*100,2))+'%'

In [ ]:
fig = px.bar(df[featstr].max(), title='Features Max Values')
fig.layout.xaxis.tickangle = 300
fig.update_traces(showlegend = False)
fig.layout.xaxis. dtick = 5
fig.layout.xaxis.title = ''
fig.layout.yaxis.title = ''
fig.show()

#### Now we make a boxplot grid again but with customized .1% : 99.9% whiskers.

In [ ]:
fig = plt.figure(figsize=(20,80))
grid =  gridspec.GridSpec(29,4,figure=fig,hspace=.5,wspace=.05)
counter = 0
for i in range(29):
    for j in range(4):
        subf = fig.add_subplot(grid[i, j]);
        sns.boxplot(x= df[featstr[counter]],saturation=.5,color= 'blue', ax= subf,width=.5,whis=(.1,99.9));
        subf.set_xlabel('')
        subf.set_title('{}'.format(featstr[counter]),fontsize=14)
        counter += 1
        gc.collect();

> ### The dataset still has many obvious outliers especially negative values since we only removed positive outliers so features which used to have symmetrical outliers now have some kind of left skewed outliers.
>### comparing both of the boxplots (before and after removing positive outliers) we can notice features from feature_3 to feature_40 which used to have symmetrical outliers now changed to have extreme negative outliers after trimming



### now we do some manual outlier trimming to these features

In [ ]:
for i,j in zip(featstr[1:][2:34],n001[2:34]):
    df = df[df[i] > j]
    gc.collect();

### now we find the total observations data loss in cleaning so far

In [ ]:
str(round(((org_len - len(df))/org_len)*100,2))+'%'

### Now we Check the histograms of features

In [ ]:
df.iloc[:,7:-2].hist(bins=100,figsize=(20,74),layout=(29,4));

> ### We can now see the following:
- the histograms of features now have way less outliers and more formal distribution
- We can also see that some features like 1, 2, 85, 87, 88 and 91 have many values falling under 0
- Some features like 49, 50, 51, 55, 56, 57, 58, and 59 still have many positive outliers

### Now we explore the relation between features and resp

In [ ]:
respcorr =  pd.Series([ df.resp.corr(df[i]) for i in featstr],index=featstr)

In [ ]:
fig = px.bar(respcorr,color = respcorr, color_continuous_scale=['red','blue'], title= 'Features Correlation with Resp')
fig.layout.xaxis.tickangle = 300
fig.layout.xaxis. dtick = 5
fig.layout.xaxis.title = ''
fig.layout.yaxis.title = 'pearson correlation'
fig.update(layout_coloraxis_showscale=False)
fig.show();

> ### We can see that features are not really correlated to Resp

### Features and Weight

#### First we make a dataframe of weight correlation with feature

In [ ]:
wecorr = []
for i in featstr:
    wecorr.append((df[df.weight != 0].weight.corr(df[df.weight != 0][i])))
wecorr = pd.DataFrame(wecorr,index=featstr)

In [ ]:
wecorr.head(2)

In [ ]:
wecorr[wecorr[0] == wecorr[0].min()]

In [ ]:
wecorr[wecorr[0] == wecorr[0].max()]

In [ ]:
fig = px.bar(wecorr,title= 'Features Correlation with Weight (no including zero weights)')
fig.layout.xaxis.tickangle = 300
fig.layout.xaxis. dtick = 5
fig.layout.xaxis.title = ''
fig.layout.yaxis.title = 'pearson correlation'
fig.update(layout_coloraxis_showscale=False)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
fig = plt.figure(figsize=(8,6))
sns.scatterplot(df[df.weight != 0].weight,df[df.weight != 0].feature_51, color = 'blue', alpha=.4)
plt.xlabel('Weight',fontsize=14)
plt.ylabel('Featre_51',fontsize=14)
plt.title('Feature_51 vs Weight\nCorrelation = {}%'.format(round(df[df.weight != 0].weight.corr(df[df.weight != 0].feature_51),4)*100),fontsize=18);

> ### It seems that weight is highly correlated to feature 51

In [ ]:
fig = plt.figure(figsize=(8,6))
sns.scatterplot(df[df.weight != 0].weight,df[df.weight != 0].feature_126, color = 'blue', alpha=.4)
plt.xlabel('Weight',fontsize=14)
plt.ylabel('Featre_126',fontsize=14)
plt.title('Feature_126 vs Weight\nCorrelation{}%'.format(round(df[df.weight != 0].weight.corr(df[df.weight != 0].feature_126),4)*100),fontsize=18);

> ### While there is some kind negative correlation between weight and feature 126 the relation seem to be weak

<a id='f0'></a>
## **Feature 0**

#### Finding the unique values of Feature_0

In [ ]:
df.feature_0.unique()

In [ ]:
plt.figure(figsize=(7,5)) 
df.feature_0.value_counts().plot.bar(color='darkblue',alpha=.6,width=.5)
plt.title('Feature_0',fontsize=18) 
plt.xticks(rotation=0,fontsize=14) ;

> ### Feature_0 seem to be some kind of binary feature

In [ ]:
plt.figure(figsize=(8,6)) 
sns.countplot(data=df, x='feature_0', hue='action',palette='viridis')
plt.legend(ncol=1, fontsize=12, loc=3,title= 'Action',title_fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=13)
plt.xlabel('Feature 0',fontsize=12)
plt.title('Feature 0 and Action', fontsize=18)
plt.ylabel('')
plt.xlim(-1,2);

> ### seems like there is not an obvious relation between feature_0 and resp being negative or positive

# <font color='green'>Work in progress  █████████▒▒</font> 

<h3><center>Upvote this notebook and Michelangelo will get his favorite pizza</center></h3>

![](http://i.imgflip.com/1ydu71.gif)